---
# Data Preparation (Face Alignment and Augmentation):-
---

### <u>Landmarks And Crop Face:</u>

In [1]:
import cv2
import dlib
import numpy as np
import os
import json

# Paths to the directories
input_directory = 'Images/train_im/training' 
output_directory = 'cropped_faces'
landmarks_directory = 'landmarks'
failed_images_log = 'failed_images.txt'

# Create the output directories if they don't exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(landmarks_directory, exist_ok=True)

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()

predictor_path = "shape_predictor_68_face_landmarks.dat"  
if not os.path.isfile(predictor_path):
    raise FileNotFoundError(f"The file {predictor_path} does not exist.")
predictor = dlib.shape_predictor(predictor_path)

def detect_face_and_landmarks(image_path):
# Detect faces and landmarks in an image and return the coordinates.
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load image: {image_path}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    landmarks_list = []
# If no faces are detected, return an empty list
    if len(faces) == 0:
        print(f"No faces detected in {image_path}. Skipping this image.")
        return [], []
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks_list.append([(p.x, p.y) for p in landmarks.parts()])
    return faces, landmarks_list

def crop_and_save_face(image, face, output_path):
# Crop the face from the image and save it.
    x, y, w, h = (face.left(), face.top(), face.width(), face.height())
    face_img = image[y:y+h, x:x+w]
    cv2.imwrite(output_path, face_img)

# Process each image in the input directory
for filename in os.listdir(input_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(input_directory, filename)
        try:
            faces, landmarks_list = detect_face_and_landmarks(image_path)
        except ValueError as ve:
            print(ve)
            with open(failed_images_log, "a") as f:
                f.write(f"{filename}: Failed to load image.\n")
            continue  # Skip to the next image if there's an issue loading it

        if len(faces) == 0:
            with open(failed_images_log, "a") as f:
                f.write(f"{filename}: No faces detected.\n")
            continue  # Skip images where no faces are detected
        
        img = cv2.imread(image_path)

        for i, face in enumerate(faces):
# Save the cropped face
            face_output_path = os.path.join(output_directory, f"{filename.split('.')[0]}_face_{i}.jpg")
            try:
                crop_and_save_face(img, face, face_output_path)
            except cv2.error as e:
                print(f"Error saving face for {filename}: {e}")
                with open(failed_images_log, "a") as f:
                    f.write(f"{filename}: Failed to save cropped face.\n")
                continue

# Save the landmarks
            landmarks_output_path = os.path.join(landmarks_directory, f"{filename.split('.')[0]}_face_{i}_landmarks.json")
            try:
                with open(landmarks_output_path, 'w') as landmarks_file:
                    json.dump(landmarks_list[i], landmarks_file)
            except Exception as e:
                print(f"Error saving landmarks for {filename}: {e}")
                with open(failed_images_log, "a") as f:
                    f.write(f"{filename}: Failed to save landmarks.\n")

# Free memory after processing each image
        del img

print("Data preparation completed.")

No faces detected in Images/train_im/training\000003.jpg. Skipping this image.
No faces detected in Images/train_im/training\000004.jpg. Skipping this image.
No faces detected in Images/train_im/training\000036.jpg. Skipping this image.
No faces detected in Images/train_im/training\000067.jpg. Skipping this image.
No faces detected in Images/train_im/training\000120.jpg. Skipping this image.
No faces detected in Images/train_im/training\000137.jpg. Skipping this image.
No faces detected in Images/train_im/training\000149.jpg. Skipping this image.
No faces detected in Images/train_im/training\000150.jpg. Skipping this image.
No faces detected in Images/train_im/training\000159.jpg. Skipping this image.
No faces detected in Images/train_im/training\000166.jpg. Skipping this image.
No faces detected in Images/train_im/training\000167.jpg. Skipping this image.
No faces detected in Images/train_im/training\0001_01 (23).jpg. Skipping this image.
No faces detected in Images/train_im/training\

### <u>Face Alignment and Augmentation:</u>

In [3]:
def load_landmarks(file_path):
# Load landmarks from a JSON file.
    with open(file_path, 'r') as f:
        landmarks = json.load(f)
    return landmarks

def align_face(face, landmarks):
# Select 5 key landmarks (left eye, right eye, nose tip, left mouth corner, right mouth corner)
    key_landmarks = [
        landmarks[36],  # Left eye outer corner
        landmarks[45],  # Right eye outer corner
        landmarks[30],  # Nose tip
        landmarks[48],  # Left mouth corner
        landmarks[54]   # Right mouth corner
    ]
    
    src_points = np.array(key_landmarks, dtype=np.float32)
    
# Target points for alignment based on the desired output face size
    dst_points = np.array([[30, 30], [70, 30], [50, 50], [30, 70], [70, 70]], dtype=np.float32)
    
# Ensure that src_points and dst_points have the correct shape and contain at least 3 points
    if src_points.shape != dst_points.shape or len(src_points) < 3:
        print("Landmark shape mismatch or insufficient points")
        return None
    
# Calculate the affine transformation matrix
    matrix, _ = cv2.estimateAffinePartial2D(src_points, dst_points)
    
    if matrix is None:
        print("Failed to estimate affine matrix")
        return None
    
# Align the face using the affine transformation matrix
    aligned_face = cv2.warpAffine(face, matrix, (face.shape[1], face.shape[0]), flags=cv2.INTER_LINEAR)
    
    return aligned_face

def augment_image(image):
# Random horizontal flip
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)
    
# Random rotation
    angle = np.random.uniform(-30, 30)
    center = (image.shape[1] // 2, image.shape[0] // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    
# Random scaling
    scale = np.random.uniform(0.8, 1.2)
    matrix[0, 0] *= scale
    matrix[1, 1] *= scale
    image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    
    return image

def process_cropped_faces(cropped_faces_dir, landmarks_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(cropped_faces_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            face_path = os.path.join(cropped_faces_dir, filename)
            landmarks_path = os.path.join(landmarks_dir, f"{os.path.splitext(filename)[0]}_landmarks.json")
            
            if os.path.isfile(landmarks_path):
                face = cv2.imread(face_path)
                landmarks = load_landmarks(landmarks_path)
                
                if face is not None and landmarks:
# Ensure landmarks contain 68 points
                    if len(landmarks) == 68:
                        aligned_face = align_face(face, landmarks)
                        
                        if aligned_face is not None:
                            augmented_face = augment_image(aligned_face)
                            output_path = os.path.join(output_dir, filename)
                            cv2.imwrite(output_path, augmented_face)
                    else:
                        print(f"Incorrect number of landmarks in {landmarks_path}")
                        
cropped_faces_dir = 'cropped_faces'
landmarks_dir = 'landmarks'
output_dir = 'Processed_Faces'

process_cropped_faces(cropped_faces_dir, landmarks_dir, output_dir)

print("Data preparation completed.")

Data preparation completed.


### Data Organizing:

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split
# Paths
dataset_path = 'Processed_Faces'
train_dir = 'Processed_Images/train/'
val_dir = 'Processed_Images/val'
test_dir = 'Processed_Images/test'

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all files in the Processed_Images
all_files = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]

# Split dataset
train_files, temp_files = train_test_split(all_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

# Function to move files
def move_files(file_list, dest_folder):
    for file in file_list:
        shutil.move(os.path.join(dataset_path, file), os.path.join(dest_folder, file))

# Move files
move_files(train_files, train_dir)
move_files(val_files, val_dir)
move_files(test_files, test_dir)

In [7]:
def organize_images(base_dir, output_dir):
    for split in ['train', 'val', 'test']:
        split_dir = os.path.join(base_dir, split)
        output_split_dir = os.path.join(output_dir, split)

        # Create output directories for classes if they don't exist
        if not os.path.exists(output_split_dir):
            os.makedirs(output_split_dir)

        # Iterate through images in the split directory
        for file_name in os.listdir(split_dir):
            if file_name.endswith(('.jpg', '.jpeg', '.png')):
                # Extract the class from the filename
                # Adjust the logic if class names are different
                class_name = file_name.split('_')[1]  # e.g., extracting 'Class_A' from '000001_Class_A.jpg'
                
                # Create the class directory if it doesn't exist
                class_dir = os.path.join(output_split_dir, class_name)
                if not os.path.exists(class_dir):
                    os.makedirs(class_dir)
                
                # Move the image to the class directory
                src = os.path.join(split_dir, file_name)
                dst = os.path.join(class_dir, file_name)
                shutil.move(src, dst)
                print(f"Moved {file_name} to {class_dir}")

# Define base directory where images are located
base_dir = 'Processed_Images'
output_dir = 'Organized_Images'  # New directory to hold the organized images

# Run the function
organize_images(base_dir, output_dir)

print("Data preparation and organization completed.")

Data preparation and organization completed.


---
# Data Generation:-
----

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Parameters
IMG_HEIGHT = 32
IMG_WIDTH = 32
BATCH_SIZE = 64

train_dir = 'Organized_Images/train'
val_dir = 'Organized_Images/val'
test_dir = 'Organized_Images/test'


# Data preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # For multi-class classification
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 156260 images belonging to 87 classes.
Found 19532 images belonging to 72 classes.
Found 19533 images belonging to 71 classes.


---
# Model Architecture:-
---

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Reshape, Flatten, Dense
from tensorflow.keras.models import Sequential, load_model

# GAN Architecture
def build_generator():
    model = Sequential([
        Dense(8 * 8 * 256, input_shape=(100,)),
        LeakyReLU(),
        BatchNormalization(),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(),
        BatchNormalization(),
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(),
        Conv2DTranspose(3, (5, 5), activation='sigmoid', padding='same')
    ])
    return model

def build_discriminator():
    model = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        LeakyReLU(),
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_gan(generator, discriminator):
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

# Training
def train_gan(generator, discriminator, gan, train_generator, val_generator, epochs=10, batch_size=64):
    for epoch in range(epochs):
        for batch_images in train_generator:
            noise = np.random.normal(0, 1, (batch_size, 100))
            gen_images = generator.predict(noise)

            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))

# Train discriminator
            d_loss_real = discriminator.train_on_batch(batch_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(gen_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

# Train generator
            noise = np.random.normal(0, 1, (batch_size, 100))
            g_loss = gan.train_on_batch(noise, real_labels)

        print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss[0]}, D Accuracy: {d_loss[1]}, G Loss: {g_loss}")

# Validation
        val_loss = 0
        val_steps = 0
        for val_images in val_generator:
            noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
            gen_images = generator.predict(noise)
            val_loss += discriminator.evaluate(gen_images, np.zeros((BATCH_SIZE, 1)), verbose=0)[0]
            val_steps += 1
        val_loss /= val_steps
        print(f"Validation Loss after Epoch {epoch + 1}: {val_loss}")
        
# Save the model at checkpoints
        if (epoch + 1) % 10 == 0:
            generator.save(f'generator_epoch_{epoch + 1}.h5')
            discriminator.save(f'discriminator_epoch_{epoch + 1}.h5')
            gan.save(f'gan_epoch_{epoch + 1}.h5')

# Initialize models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Train the model
train_gan(generator, discriminator, gan, train_generator, val_generator)

# Save the final models
generator.save('generator_final.h5')
discriminator.save('discriminator_final.h5')
gan.save('gan_final.h5')

C:\Users\Hira KZ\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Hira KZ\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 187ms/step


C:\Users\Hira KZ\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
2/2 ━━━━━

### <u>Load Final Model:</u>

In [ ]:
generator = load_model('generator_final.h5')
discriminator = load_model('discriminator_final.h5')
gan = load_model('gan_final.h5')

# Test on the test set
test_loss = 0
test_steps = 0
for test_images in test_generator:
    noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
    gen_images = generator.predict(noise)
    test_loss += discriminator.evaluate(gen_images, np.zeros((BATCH_SIZE, 1)), verbose=0)[0]
    test_steps += 1
test_loss /= test_steps
print(f"Test Loss: {test_loss}")

### <u>save some generated images to evaluate visually:</u>

In [ ]:
import matplotlib.pyplot as plt

def save_generated_images(generator, num_images=5, save_dir='generated_images'):
    os.makedirs(save_dir, exist_ok=True)
    noise = np.random.normal(0, 1, (num_images, 100))
    generated_images = generator.predict(noise)
    for i in range(num_images):
        plt.imshow(generated_images[i])
        plt.axis('off')
        plt.savefig(os.path.join(save_dir, f'image_{i}.png'))

save_generated_images(generator)

---
# Face Swapping In Image:-
---

In [ ]:
def extract_face(image, face_cascade):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]
    return image[y:y+h, x:x+w]

def swap_faces(source_face, target_image, generator):
    # Preprocess source_face and target_image
    source_face = cv2.resize(source_face, (128, 128))
    target_image = cv2.resize(target_image, (128, 128))
    source_face = (source_face / 255.0 - 0.5) / 0.5
    target_image = (target_image / 255.0 - 0.5) / 0.5

    # Generate new face
    noise = np.random.normal(0, 1, (1, 100))
    generated_face = generator.predict(noise)[0]

    # Postprocess and replace face in the target image
    generated_face = ((generated_face + 1) * 127.5).astype(np.uint8)
    target_image[0:128, 0:128] = generated_face

    return target_image

# Load models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
generator = tf.keras.models.load_model('generator_final.h5')

# Load and process images
source_image = cv2.imread('Sourse_2_image.jpg')
target_image = cv2.imread('Target_image.jpg')

source_face = extract_face(source_image, face_cascade)
if source_face is not None:
    result_image = swap_faces(source_face, target_image, generator)
    cv2.imwrite('swapped_face_result.jpg', result_image)
else:
    print("No face detected in the source image.")